Retriever And Chain With Langchain 


In [1]:
from langchain_community.document_loaders import PyPDFLoader 
loader = PyPDFLoader("attention.pdf")
docs = loader.load()
docs


[Document(metadata={'producer': 'PyPDF2', 'creator': 'PyPDF', 'creationdate': '', 'subject': 'Neural Information Processing Systems http://nips.cc/', 'publisher': 'Curran Associates, Inc.', 'language': 'en-US', 'created': '2017', 'eventtype': 'Poster', 'description-abstract': 'The dominant sequence transduction models are based on complex recurrent orconvolutional neural networks in an encoder and decoder configuration. The best performing such models also connect the encoder and decoder through an attentionm echanisms.  We propose a novel, simple network architecture based solely onan attention mechanism, dispensing with recurrence and convolutions entirely.Experiments on two machine translation tasks show these models to be superiorin quality while being more parallelizable and requiring significantly less timeto train. Our single model with 165 million parameters, achieves 27.5 BLEU onEnglish-to-German translation, improving over the existing best ensemble result by over 1 BLEU. On 

In [3]:
pip install -U langchain-text-splitters

Note: you may need to restart the kernel to use updated packages.


In [4]:
# 1. New import path (note the 's' at the end of splitters)
from langchain_text_splitters import RecursiveCharacterTextSplitter 

# 2. Use the full class name: RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=20)

# 3. This will now work!
chunks = text_splitter.split_documents(docs)
print(chunks[:5])

[Document(metadata={'producer': 'PyPDF2', 'creator': 'PyPDF', 'creationdate': '', 'subject': 'Neural Information Processing Systems http://nips.cc/', 'publisher': 'Curran Associates, Inc.', 'language': 'en-US', 'created': '2017', 'eventtype': 'Poster', 'description-abstract': 'The dominant sequence transduction models are based on complex recurrent orconvolutional neural networks in an encoder and decoder configuration. The best performing such models also connect the encoder and decoder through an attentionm echanisms.  We propose a novel, simple network architecture based solely onan attention mechanism, dispensing with recurrence and convolutions entirely.Experiments on two machine translation tasks show these models to be superiorin quality while being more parallelizable and requiring significantly less timeto train. Our single model with 165 million parameters, achieves 27.5 BLEU onEnglish-to-German translation, improving over the existing best ensemble result by over 1 BLEU. On 

In [5]:
documents=text_splitter.split_documents(docs)
documents

[Document(metadata={'producer': 'PyPDF2', 'creator': 'PyPDF', 'creationdate': '', 'subject': 'Neural Information Processing Systems http://nips.cc/', 'publisher': 'Curran Associates, Inc.', 'language': 'en-US', 'created': '2017', 'eventtype': 'Poster', 'description-abstract': 'The dominant sequence transduction models are based on complex recurrent orconvolutional neural networks in an encoder and decoder configuration. The best performing such models also connect the encoder and decoder through an attentionm echanisms.  We propose a novel, simple network architecture based solely onan attention mechanism, dispensing with recurrence and convolutions entirely.Experiments on two machine translation tasks show these models to be superiorin quality while being more parallelizable and requiring significantly less timeto train. Our single model with 165 million parameters, achieves 27.5 BLEU onEnglish-to-German translation, improving over the existing best ensemble result by over 1 BLEU. On 

In [6]:

from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import OllamaEmbeddings

embeddings = OllamaEmbeddings(model="llama2")

db = FAISS.from_documents(documents[:20],OllamaEmbeddings())

#print("FAISS vector store created successfully!")

C:\Users\vishu\AppData\Local\Temp\ipykernel_33228\4148142033.py:4: LangChainDeprecationWarning: The class `OllamaEmbeddings` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the `langchain-ollama package and should be used instead. To use it run `pip install -U `langchain-ollama` and import as `from `langchain_ollama import OllamaEmbeddings``.
  embeddings = OllamaEmbeddings(model="llama2")


In [7]:
db

In [8]:
query = "Scaled Dot-Product Attention"
#result = db.similarity_search(query,k=3)
#print(result[0].page_content)
result = db.max_marginal_relevance_search(query, k=3)
#print(result[0].page_content)
print(result[1].page_content)

Since our model contains no recurrence and no convolution, in order for the model to make use of the
order of the sequence, we must inject some information about the relative or absolute position of the
tokens in the sequence. To this end, we add "positional encodings" to the input embeddings at the
5


In [10]:
pip install -U langchain-ollama


Note: you may need to restart the kernel to use updated packages.


In [12]:
from langchain_ollama import OllamaLLM
#from langchain_community.llms import Ollama
## Load Ollama LAMA2 LLM Model 
llm=Ollama(model="llama2")
llm

Ollama()

In [13]:
## Design ChatPrompt Template
from langchain_core.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_template("""Answer the following question based only on the provided context.
Think the following questions based only on the provided context.
Think step by step before providing a detailed answer.
I will tip you $1000 if the user finds the answer helpful.
<context>
{context}
</context>
Question: {input}""")

In [18]:
%pip install -U langchain

In [22]:
pip install langchain-classic

In [ ]:
# Chain Introduction 
# Create Stuff Document Chain
# Change 'langchain.chains' to 'langchain_classic.chains'
from langchain_classic.chains.combine_documents import create_stuff_documents_chain

# Note the lowercase 's' - Python is case-sensitive!
document_chain = create_stuff_documents_chain(llm, prompt)

In [24]:
"""
Retrievers: A retriever is an interface that returns documents given an unstructured query. It is more general than a vector store.
A retriever does not need to be able to store documents,only to return or retrieve them. Vector stores can be used as the backbone of a retriever, but there are other types of retrievers as well.
https://python.langchain.com/docs/modules/data_connection/retrievers/
"""

retriever=db.as_retriever()
retriever

VectorStoreRetriever(tags=['FAISS', 'OllamaEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x0000028A5FDBC580>, search_kwargs={})

In [25]:
""" 
Retrieval chain: This chain takes in a user inquiry,which is then passed to the retriever to fetch relevant documents.Those documents (and original inputs) are then passed to an LLM to generate a response
https://python.langchain.com/docs/modules/chains/
"""
from langchain_classic.chains import create_retrieval_chain
retrieval_chain=create_retrieval_chain(retriever,document_chain)



In [29]:
response=retrieval_chain.invoke({"input":"An attention function can be described as mapping as query"})

In [30]:
response["answer"]

"The attention function in the context you provided is Scaled Dot-Product Attention. The attention function takes three inputs: queries, keys, and values. The queries and keys are vectors of dimension dk, while the values are vectors of dimension dv. The output of the attention function is a vector of dimension dk, where each element is computed as a weighted sum of the values, using a compatibility function of the query with the corresponding key. In Scaled Dot-Product Attention, the weights are computed as the dot product of the query and key divided by the square root of the key's dimensionality."